In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing, metrics
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_pickle('train.pkl')
train.shape

(21226172, 74)

Because of the size of our data, we will be using LGB to do our modeling and feature selection.

In [3]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,rolling_skew_t30,rolling_kurt_t30,year,quarter,month,week,dayofweek,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend,bollinger,rrg_bench,rrg_item,ATR,ann_vol,entropy,beta,info_ratio,triple_exp,relative_vol,RS,gap_size,gap_start,number_of_gaps,rolling_max_14,rolling_max_180,rolling_max_30,rolling_max_60,rolling_max_90,rolling_mean_14,rolling_mean_180,rolling_mean_30,rolling_mean_60,rolling_mean_90,rolling_min_14,rolling_min_180,rolling_min_30,rolling_min_60,rolling_min_90,rolling_std_14,rolling_std_180,rolling_std_30,rolling_std_60,rolling_std_90,sales_lag_15,sales_lag_17,sales_lag_19,sales_lag_21,sales_lag_23,sales_lag_25,sales_lag_27,sales_lag_29,sales_lag_31,sales_lag_33,sales_lag_35,event,gap_placement
0,HOUSEHOLD_1_030_CA_4_validation,2031,5,2,3,0,0,2014-05-14,11415,0,0,1,4.968750,0.0,0.0,2014,2,5,20,2,0,0,0,0,0,0,0,0.0,5.0,5.0,3.0,18.234375,1.559570,0.0,0.000865,0.120850,0.0,0.0,921.0,4.0,1.0,1.0,4.0,2.0,3.0,4.0,0.428467,0.688965,0.466553,0.566895,0.677734,0.0,0.0,0.0,0.0,0.0,0.513672,0.976074,0.628906,0.789062,0.897461,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,0,0.0
1,HOUSEHOLD_1_032_CA_4_validation,2032,5,2,3,0,2,2014-05-14,11415,0,0,1,10.859375,0.0,0.0,2014,2,5,20,2,0,0,0,0,0,0,0,2.0,5.0,5.0,3.0,18.234375,2.394531,0.0,0.021866,1.757812,0.0,0.0,921.0,4.0,1.0,5.0,5.0,5.0,5.0,5.0,1.286133,0.655762,1.566406,1.383789,1.311523,0.0,0.0,0.0,0.0,0.0,1.772461,1.154297,1.612305,1.290039,1.346680,1.0,5.0,2.0,1.0,1.0,3.0,1.0,1.0,3.0,1.0,1.0,0,0.0
2,HOUSEHOLD_1_033_CA_4_validation,2033,5,2,3,0,0,2014-05-14,11415,0,0,1,4.468750,0.0,0.0,2014,2,5,20,2,0,0,0,0,0,0,0,0.0,5.0,5.0,3.0,18.234375,1.156250,0.0,-0.008560,0.000000,0.0,0.0,921.0,4.0,1.0,0.0,2.0,0.0,2.0,2.0,0.000000,0.183350,0.000000,0.133301,0.177734,0.0,0.0,0.0,0.0,0.0,0.000000,0.441895,0.000000,0.389160,0.438965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,HOUSEHOLD_1_034_CA_4_validation,2034,5,2,3,0,1,2014-05-14,11415,0,0,1,3.970703,0.0,0.0,2014,2,5,20,2,0,0,0,0,0,0,0,1.0,5.0,5.0,3.0,18.234375,1.230469,0.0,-0.007103,0.878906,0.0,0.0,921.0,4.0,1.0,2.0,5.0,5.0,5.0,5.0,0.285645,0.438965,0.566895,0.500000,0.444336,0.0,0.0,0.0,0.0,0.0,0.611328,0.799316,1.194336,0.947754,0.900879,0.0,0.0,1.0,0.0,4.0,1.0,0.0,5.0,0.0,0.0,0.0,0,0.0
4,HOUSEHOLD_1_035_CA_4_validation,2035,5,2,3,0,0,2014-05-14,11415,0,0,1,4.628906,0.0,0.0,2014,2,5,20,2,0,0,0,0,0,0,0,0.0,5.0,5.0,3.0,18.234375,0.443115,0.0,-0.018326,0.000000,0.0,0.0,921.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.000000,0.005554,0.000000,0.000000,0.011108,0.0,0.0,0.0,0.0,0.0,0.000000,0.074524,0.000000,0.000000,0.105408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [ ]:
def run_lgb(data):
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()
    
    params = {
#         'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'poisson',
        'n_jobs': -1,
        'seed': 20,
        'learning_rate': 0.1,
        'alpha': 0.1,
        'lambda': 0.1,
        'bagging_fraction': 0.66,
        'bagging_freq': 2, 
        'colsample_bytree': 0.77}

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    
    del x_train, y_train
    
    
    model = lgb.train(params, train_set, num_boost_round = 2000, early_stopping_rounds = 200, valid_sets = [train_set, val_set], verbose_eval = 100)
    joblib.dump(model, 'lgbm_0.sav')
    
    val_pred = model.predict(x_val[features], num_iteration=model.best_iteration)
    val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score}')
    y_pred = model.predict(test[features], num_iteration=model.best_iteration)
    test['demand'] = y_pred
    return test

def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)
    
def transform_train_and_eval(data):
    # reduce memory for new features so we can train
    data = reduce_mem_usage(data)
    test = run_lgb(data)
    predict(test, submission_df)